### Import Required Package

In [5]:
import pandas as pd
import talib

import json

### Import Tidal

In [6]:
import tidal as td

### Initialize Plumber

In [7]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [8]:
import pandas as pd
import os
import numpy as np
os.chdir('/home/jovyan/individualized-indicator')
test_1 = pd.read_pickle('./emb/2023_03_01/top_8_2023-06-01_2024-06-28.pkl')
df_reset = test_1.reset_index()
unique_values, indices, counts = np.unique(df_reset.iloc[:,0], return_index=True, return_counts=True)
sorted_unique_values = unique_values[np.argsort(indices)]
modified_arr = np.array(sorted_unique_values)
# df_reset.iloc[:,0] = np.array([s[:-3] for s in df_reset.iloc[:,0]]) 
df_reset.iloc[:,0] = np.array(df_reset.iloc[:,0])
multi_index_df = df_reset.set_index(['instrument', 'datetime'])
multi_index_df = multi_index_df.dropna()

DEFAULT_STOCKS = list(modified_arr.astype(object))
start_date = "2023-06-01"
end_date = "2024-06-28"

quote_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in DEFAULT_STOCKS]),
    },
)
quote_data.index.set_levels(
    pd.to_datetime(quote_data.index.levels[1]),
    level=1,
    inplace=True,
)
quote_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

instruments = quote_data.index.get_level_values('instrument').unique()

quote_data['new'] = multi_index_df.iloc[:,5]

quote_data

/tmp/ipykernel_14457/3292540928.py:28: FutureWarning: inplace is deprecated and will be removed in a future version.
  quote_data.index.set_levels(


open      high       low     close    volume  \
instrument datetime                                                       
1101       2023-06-01   36.4362   36.4841   36.1489   36.1489  20325434   
           2023-06-02   36.1489   37.1065   36.1489   36.9150  37641391   
           2023-06-05   37.1065   37.5853   37.0108   37.2023  23588849   
           2023-06-06   37.2980   37.3938   37.0108   37.1544  11784017   
           2023-06-07   37.2502   37.2980   36.9629   37.1544  11693000   
...                         ...       ...       ...       ...       ...   
9958       2024-06-24  331.3057  332.2918  317.5013  324.4035  10111772   
           2024-06-25  325.8826  331.7988  317.5013  320.9524   7282478   
           2024-06-26  321.4455  331.3057  321.4455  324.8966   5833060   
           2024-06-27  322.4315  329.3337  319.9664  320.4594   3860452   
           2024-06-28  320.9524  321.9385  290.8786  297.7808  20309330   

                            new  
instrument datetime              
1101       2023-06-01  0.000015  
           2023-06-02  0.000188  
           2023-06-05  0.000122  
           2023-06-06  0.000065  
           2023-06-07  0.000075  
...                         ...  
9958       2024-06-24  0.000150  
           2024-06-25  0.000184  
           2024-06-26  0.000377  
           2024-06-27  0.000221  
           2024-06-28  0.000072  

[95469 rows x 6 columns]

### Load benchmark data

In [9]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_14457/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2023-06-01  120.1166  120.3561  119.6854  119.9249   4231123
           2023-06-02  120.5478  121.6977  120.5478  121.4581   8412015
           2023-06-05  121.4581  121.5060  120.9790  121.0748   6813168
           2023-06-06  121.0748  121.9373  120.8832  121.4102   6877372
           2023-06-07  121.6977  122.8955  121.6977  122.7518  12723827
...                         ...       ...       ...       ...       ...
           2024-06-24  185.5518  185.5518  182.6168  183.0645  14671365
           2024-06-25  181.0250  182.9651  179.8808  182.9651  13774562
           2024-06-26  184.0097  185.1539  183.3133  184.4077   9269483
           2024-06-27  182.6666  184.3579  182.4676  184.3579   6508242
           2024-06-28  184.3579  186.4473  183.7112  185.5021   4455951

[263 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [10]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=1000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi")

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [11]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [12]:
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05}}

### Config Modification

In [13]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [14]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [15]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 263/263 [00:08<00:00, 30.28it/s, cash=5.6e+4, pnl=1.37e+5, position_cost=1.04e+6, value=1.23e+6] 


### Metric - AccountInfo

In [16]:
tidal.metrics['AccountInfo'].report

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown,turnover
datetime,,,,,,,,
2023-06-01,1000000.000000,0.000000,0.000000e+00,0.0000,0.0000,1.000000e+06,0.000000,0.0000
2023-06-02,183145.750337,267.636263,8.165866e+05,4357.2134,-3957.5134,9.957749e+05,-0.004225,816586.6134
2023-06-05,183145.750337,267.636263,8.165866e+05,4357.2134,4018.9866,1.003751e+06,0.000000,0.0000
2023-06-06,42275.583617,979.002983,9.526090e+05,6874.0134,7238.0866,1.002123e+06,-0.001623,504229.2000
2023-06-07,42537.340878,1646.529322,9.460979e+05,7602.9298,38948.8828,1.027584e+06,0.000000,364454.8836
...,...,...,...,...,...,...,...,...
2024-06-24,50142.078786,101064.415214,1.052330e+06,110265.6060,69604.0000,1.172076e+06,-0.222059,0.0000
2024-06-25,50142.078786,101064.415214,1.052330e+06,110265.6060,94329.6000,1.196801e+06,-0.205648,0.0000
2024-06-26,50142.078786,101064.415214,1.052330e+06,110265.6060,126964.6000,1.229436e+06,-0.183987,0.0000


### Metric - PositionInfo

In [17]:
tidal.metrics['PositionInfo'].report

quantity      price  commission  slip_cost        pnl  \
instrument datetime                                                            
3029       2023-06-02    3000.0  62.757965   61.706769  1514.5962   -91.0962   
           2023-06-05    3000.0  62.757965   61.706769  1514.5962 -2083.9962   
           2023-06-06    3000.0  62.757965   61.706769  1514.5962 -5215.6962   
           2023-06-08    2000.0  60.856169   39.891219   242.9388 -4038.7388   
           2023-06-14    3000.0  59.049563   58.060483   922.9902   500.5098   
...                         ...        ...         ...        ...        ...   
8420       2024-06-21    3000.0  44.200000   43.459650   300.0000  -600.0000   
           2024-06-24    3000.0  44.200000   43.459650   300.0000 -2400.0000   
           2024-06-25    3000.0  44.200000   43.459650   300.0000 -2400.0000   
           2024-06-26    3000.0  44.200000   43.459650   300.0000 -1950.0000   
           2024-06-27    3000.0  44.200000   43.459650   300.0000 -2400.0000   

                       now_price  
instrument datetime               
3029       2023-06-02    62.7276  
           2023-06-05    62.0633  
           2023-06-06    61.0194  
           2023-06-08    58.8368  
           2023-06-14    59.2164  
...                          ...  
8420       2024-06-21    44.0000  
           2024-06-24    43.4000  
           2024-06-25    43.4000  
           2024-06-26    43.5500  
           2024-06-27    43.4000  

[1310 rows x 6 columns]

In [18]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [19]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2023-06-01  0.000015
           2023-06-02  0.000188
           2023-06-05  0.000122
           2023-06-06  0.000065
           2023-06-07  0.000075
...                         ...
9958       2024-06-24  0.000150
           2024-06-25  0.000184
           2024-06-26  0.000377
           2024-06-27  0.000221
           2024-06-28  0.000072

[95469 rows x 1 columns]

### Trade Report

In [20]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6139,2.0,0.0,2.0,1.0,1.000000,452021.196413,0.000000,396653.933387,452021.196413
3029,20.0,21.0,41.0,1.0,0.487805,112331.220070,-77082.246365,258362.727495,35248.973705
6470,9.0,9.0,18.0,1.0,0.500000,60925.206347,-70554.567904,122529.501757,-9629.361557
4557,7.0,10.0,17.0,1.0,0.411765,13751.236093,-34430.930081,122377.913188,-20679.693988
3038,4.0,7.0,11.0,1.0,0.363636,14082.248277,-56995.639562,190026.344085,-42913.391285
6570,14.0,29.0,43.0,0.0,0.325581,48495.273571,-95979.138353,26752.293782,-47483.864782
8049,3.0,9.0,12.0,0.0,0.250000,4003.145533,-64264.254750,8649.916616,-60261.109216
8420,9.0,20.0,29.0,0.0,0.310345,70079.338526,-143436.344431,16307.005904,-73357.005904


### Traded instruments

In [21]:
tidal.account.trades.keys()

dict_keys(['6139', '3038', '4557', '3029', '8420', '6570', '6470', '8049'])

### Trading History

In [22]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [23]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [24]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:46531
 * Running on http://10.136.11.26:46531
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:02] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:03] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:03] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:03] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:03] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31:03] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [26/Aug/2024 11:31: